# 数据科学-概率学

## 概率密度函数和累积分布函数

### 概率密度函数（Probability Density Function, PDF）

**概率密度函数**是用来描述连续型随机变量的概率分布的函数。它表示随机变量落在某一特定取值附近的概率密度。

**关键点：**

* **非负性：** PDF的值总是大于等于0。
* **归一性：** PDF在整个定义域上的积分等于1，即概率总和为1。
* **概率密度，而非概率：** PDF的值本身并不是概率，而是概率密度。要计算随机变量落在某一区间内的概率，需要对该区间上的PDF进行积分。

**直观理解：**
* PDF的图像可以看作是一个曲线，曲线下的面积代表概率。
* 曲线越高的地方，随机变量落在该处附近的概率越大。

**数学表示：**
```
f(x) >= 0
∫f(x)dx = 1
```

### 累积分布函数（Cumulative Distribution Function, CDF）

**累积分布函数**表示随机变量小于等于某个值的概率。它给出了随机变量取值小于等于x的概率。

**关键点：**

* **单调递增：** CDF的值随着x的增大而单调递增。
* **范围：** CDF的值在0到1之间。
* **概率：** CDF(x)表示随机变量X小于等于x的概率。

**直观理解：**
* CDF的图像是一条从0到1的曲线。
* 曲线上的每个点表示随机变量小于等于对应x值的概率。

### PDF和CDF的关系

* **CDF是PDF的积分：** CDF是PDF从负无穷大到x的积分。
* **PDF是CDF的导数：** 在PDF连续的点上，PDF是CDF的导数。

### 示例：正态分布

正态分布是一种常见的连续型概率分布。它的PDF呈钟形曲线，CDF是一条S形曲线。

## 随机采样

### **逆变换采样法**

* **原理：**
  * 计算概率密度函数的累积分布函数（CDF）。
  * 生成一个[0, 1]之间的均匀分布随机数。
  * 将这个均匀分布随机数作为CDF的输入，求解对应的x值，这个x值就是我们想要的随机数。

* **代码示例：**

```python
import numpy as np
import matplotlib.pyplot as plt

# 自定义概率密度函数
def my_pdf(x):
    return 0.5 * np.exp(-np.abs(x))

# 计算累积分布函数（假设有解析解）
def my_cdf(x):
    return 0.5 * (1 + np.sign(x) * (1 - np.exp(-np.abs(x))))

# 逆变换采样
def inverse_transform_sampling(n):
    u = np.random.uniform(size=n)
    x = np.log(2*u - 1) * np.sign(u - 0.5)
    return x

# 生成1000个随机数
samples = inverse_transform_sampling(1000)

# 绘制直方图
plt.hist(samples, bins=50, density=True)
plt.show()
```

### **拒绝采样法**

* **原理：**
  * 定义一个容易采样的分布作为提议分布。
  * 从提议分布中采样一个点。
  * 根据自定义分布和提议分布的比值，决定是否接受该采样点。
  * 重复上述过程，直到采样到足够多的点。

* **代码示例：**

```python
# 拒绝采样
def rejection_sampling(n):
    samples = []
    while len(samples) < n:
        x = np.random.randn()  # 提议分布为标准正态分布
        u = np.random.uniform(0, 1)
        if u <= my_pdf(x) / (0.4 * np.exp(-x**2/2)):
            samples.append(x)
    return np.array(samples)
```

* **对一整个数组进行拒绝采样的原理:** `np.random.choice` 函数可以从一个一维数组中随机抽取元素，每个元素被抽中的概率与其在数组中对应的值成正比。
* **用法:**
    ```python
    import numpy as np

    # 假设 probs 是一个表示概率的数组
    samples = np.random.choice(np.arange(len(probs)), size=num_samples, p=probs)
    ```
* **完整代码示例**

    ```python
    import numpy as np

    def custom_pdf(x):
        # 自定义的概率密度函数
        return np.exp(-x**2)

    # 参数设置
    min_val = -5
    max_val = 5
    num_samples = 1000
    num_candidates = 10 * num_samples  # 候选样本数量

    # 均匀随机采样候选样本
    candidates = np.linspace(min_val, max_val, num_candidates)

    # 计算每个候选样本的概率密度
    probs = custom_pdf(candidates)

    # 归一化概率
    probs /= probs.sum()

    # 使用 np.random.choice 采样
    indices = np.random.choice(np.arange(len(probs)), size=num_samples, p=probs)

    # 获取最终的样本
    samples = candidates[indices]

    # 可视化结果
    import matplotlib.pyplot as plt
    plt.hist(samples, bins=50, density=True)
    plt.show()
    ```

### **MCMC采样**

MCMC（Markov Chain Monte Carlo）是一种强大的随机采样方法，广泛应用于统计学、机器学习等领域。它能够从复杂的概率分布中生成样本，即使这个分布的概率密度函数很难直接计算。

**MCMC的基本原理**

MCMC的核心思想是构建一个马尔可夫链，使得这个马尔可夫链的平稳分布就是我们想要采样的目标分布。也就是说，经过足够多次的迭代后，马尔可夫链的状态就会收敛到目标分布。

**MCMC算法的一般步骤：**

1.  **初始化：** 随机选取一个初始状态。
2.  **迭代：**
      * 根据当前状态，产生一个新的候选状态。
      * 计算接受概率：根据目标分布和当前状态、候选状态，计算一个接受概率。
      * 决策：
          * 如果接受概率大于一个随机生成的均匀分布值，则接受候选状态作为下一个状态。
          * 否则，保留当前状态。
3.  **重复步骤2：** 重复迭代多次，直到马尔可夫链收敛。

**常用的MCMC算法**

  * **Metropolis-Hastings算法:** 这是最基础的MCMC算法，其接受概率计算方式相对简单。
  * **Gibbs采样:** 当目标分布是多个变量的联合分布时，Gibbs采样可以逐个变量进行更新，往往比Metropolis-Hastings算法更有效。
  * **Hamiltonian蒙特卡洛:** 通过引入辅助变量，将采样问题转化为模拟一个物理系统，从而提高采样效率。

**MCMC的应用**

  * **贝叶斯统计:** 在贝叶斯统计中，MCMC用于从后验分布中抽样，从而得到模型参数的估计。
  * **机器学习:** MCMC可以用于训练复杂的概率图模型，如隐马尔可夫模型、贝叶斯网络等。
  * **物理模拟:** MCMC可以用于模拟复杂的物理系统，例如统计物理中的自旋模型。

**Python实现MCMC**

Python中有多个库可以方便地实现MCMC采样，例如：

  * **PyMC3:** 一个高层次的概率编程框架，提供了丰富的MCMC工具。
  * **Stan:** 一个基于C++的高性能统计计算平台，支持MCMC采样。
  * **TensorFlow Probability:** TensorFlow的一个概率编程库，也提供了MCMC功能。

**示例：使用PyMC3进行MCMC采样**

```python
import pymc3 as pm
import numpy as np

# 假设我们想从一个正态分布中采样
with pm.Model() as model:
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd', 1)
    y = pm.Normal('y', mu=mu, sd=sd, observed=data)

    # 使用NUTS算法进行采样
    trace = pm.sample(1000, tune=1000)
```

